In [1]:
import requests, json
from pprint import pprint
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
# base url for all FPL API endpoints
base_url = 'https://fantasy.premierleague.com/api/'

# get data from bootstrap-static endpoint
r = requests.get(base_url+'bootstrap-static/').json()

# show the top level fields
pprint(r, indent=2, depth=1, compact=True)

{ 'element_stats': [...],
  'element_types': [...],
  'elements': [...],
  'events': [...],
  'game_settings': {...},
  'phases': [...],
  'teams': [...],
  'total_players': 9102227}


In [3]:
# create players dataframe
players = pd.json_normalize(r['elements'])
# create teams dataframe
teams = pd.json_normalize(r['teams'])
# get position information from 'element_types' field
positions = pd.json_normalize(r['element_types'])

In [4]:
players.head(1)

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,id,in_dreamteam,news,news_added,now_cost,photo,points_per_game,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text
0,100.0,100.0,80201,0,0,-5,5,1,1,3.5,2.0,0,Bernd,1.5,1,False,,2022-02-11T08:00:15.144286Z,45,80201.jpg,2.5,Leno,0.9,False,None,a,1,3,10,79675,155,198707,1986,0.3,2.2,Leno,360,0,0,1,9,0,0,0,0,0,10,0,69,85.0,0.0,0.0,8.5,377,29,606,69,588,60,433,30,NaN,,NaN,,NaN,


In [5]:
teams.head(1)

,code,draw,form,id,loss,name,played,points,position,short_name,strength,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,0,ARS,4,None,False,0,1250,1270,1150,1210,1190,1220,1


In [6]:
positions.head(1)

,id,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
0,1,Goalkeepers,GKP,Goalkeeper,GKP,2,1,1,True,[12],83


In [7]:
# select columns of interest from players df
players = players[
    ['id', 'first_name', 'second_name', 'web_name', 'team', 'now_cost', 'total_points',
     'element_type']
]

# join team name
df = players.merge(
    teams[['id', 'name']],
    left_on='team',
    right_on='id',
    suffixes=['_player', None]
).drop(['team', 'id'], axis=1)

# join player positions
df = df.merge(
    positions[['id', 'singular_name']],
    left_on='element_type',
    right_on='id'
).drop(['web_name', 'element_type', 'id'], axis=1)

df = df.rename(
    columns={'name':'team_name', 'singular_name':'position_name'}
)

In [8]:
df.sort_values(by='total_points', ascending = False)

,id_player,first_name,second_name,now_cost,total_points,team_name,position_name
243,233,Mohamed,Salah,132,230,Liverpool,Midfielder
323,359,Heung-Min,Son,111,198,Spurs,Midfielder
599,237,Trent,Alexander-Arnold,84,191,Liverpool,Defender
613,256,João Pedro Cavaco,Cancelo,70,173,Man City,Defender
360,420,Jarrod,Bowen,69,168,West Ham,Midfielder
...,...,...,...,...,...,...,...
235,658,Archie,Gray,45,0,Leeds,Midfielder
234,631,Jack,Jenkins,45,0,Leeds,Midfielder
550,690,Robert Kenedy,Nunes do Nascimento,45,0,Chelsea,Defender
721,686,Dion,Sanderson,40,0,Wolves,Defender
